# calculating_snr 

- https://github.com/lsst/rubin_sim_notebooks/tree/main/photometry

This ipython notebook demonstrates how to calculate photometric (optimal psf-weighted) SNR for sources observed with LSST. 

It uses the [rubin_sim.photUtils](https://github.com/lsst/rubin_sim) package, to calculate the SNR for any spectrum. Baseline LSST throughput curves come with the `rubin_sim` package, in `rubin_sim_data`.

An overview of the process (starting from the spectrum) is:
* Set the desired magnitude (in a standardized throughput curve) for the source - this sets the SED normalization.
* Generate throughput curves in all filters, for the airmass desired.
* Set the sky SED (for now we just use a dark, zenith sky SED, but can scale the overall sky brightness up or down).
* Calculate the SNR. 


In [ ]:
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from rubinsimphot.phot_utils import Bandpass
from rubinsimphot.phot_utils import Sed
#from rubin_sim.phot_utils import SignalToNoise
from rubinsimphot.phot_utils import signaltonoise
from rubinsimphot.phot_utils import PhotometricParameters
from rubinsimphot.phot_utils import calc_neff

from rubinsimphot.site_models import SeeingModel
from rubinsimphot.data import get_data_dir

-----

###  Read some useful data ###

<b>Read some sample spectra.</b> 

The repo [photometry_sample](https://github.com/rhiannonlynne/photometry_sample) contains a few SEDs that cover a fairly wide range: elliptical and spiral galaxies, blue to red stars, white dwarfs, sn, and a quasar. We'll read a subset of them into `Sed` objects, and redshift the quasar. 

Clone the repo, then set `sedDir` to whereever you place the directory.

In [ ]:
get_data_dir()

In [ ]:
sedDir = os.path.join(get_data_dir(),'photometry_sample')
sedtypes = ['galaxies', 'quasar', 'stars'] 
sedtypes_not_used = ['sn', 'photoZ_outliers', 'white_dwarf']
#sedtypes = sedtypes + sedtypes_not_used

seds = {}
for s in sedtypes:
    filenames = os.listdir(os.path.join(sedDir, s))
    for f in filenames:
        name = f.replace('.dat', '').replace('.sed', '')
        seds[name] = Sed()
        seds[name].read_sed_flambda(os.path.join(sedDir, s, f))
list(seds.keys())

In [ ]:
# Redshift the quasar. Note repeat calls to this cell will stack, increasing the redshift multiple times.
z=1.5
seds['quasar'].redshift_sed(z)

In [ ]:
# Plot the seds
plt.figure(figsize=(8,5))
for n in seds:
    plt.plot(seds[n].wavelen, seds[n].flambda * seds[n].wavelen, label=n)
plt.legend(loc=(1.01, 0.01))
plt.xlim(300, 1100)
plt.grid(alpha=0.3)
plt.xlabel('Wavelength (nm)')
plt.ylabel("Wavelength * Flambda")

<b>Read the standard throughput components, a range of atmospheres, and a sky background spectra</b>.

These are provided as part of `rubin_sim_data`, in the `throughputs` directory. 

In [ ]:
throughputsDir = os.path.join(get_data_dir(), 'throughputs')

!ls -l $throughputsDir

filterlist = ('u', 'g', 'r', 'i', 'z', 'y')
filtercolors = {'u':'c', 'g':'g', 'r':'y', 'i':'r', 'z':'m', 'y':'k'}

Read in the set of throughput curves we'll use as our "standard". These are the 'standard' LSST total throughput curves and use an X=1.2 atmosphere (with aerosols) together with the standard hardware components (with time-average losses). 

In [ ]:
lsst_std = {}
for f in filterlist:
    lsst_std[f] = Bandpass()
    lsst_std[f].read_throughput(os.path.join(throughputsDir, 'baseline', f'total_{f}.dat'))

In [ ]:
# Plot the filters 

plt.figure(figsize=(8,5))
for f in filterlist:
    plt.plot(lsst_std[f].wavelen, lsst_std[f].sb, color=filtercolors[f], label=f)
plt.legend()
plt.ylim(0, 0.5)
plt.xlim(300, 1100)
plt.grid(alpha=0.4)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Sb (0-1)")
plt.title("Standard throughput curves (X=1.2)")

Read in the zenith, dark sky spectrum. This is the only sky background spectrum we have available at this time .. obviously it's not the right colors for other lunar phases, as a function of distance from the moon, or even necessarily at different airmasses. However, as a placeholder, we can scale it up and down to match a specified sky background magnitude in a different bandpass. 

In [ ]:
darksky = Sed()
darksky.read_sed_flambda(os.path.join(throughputsDir, 'baseline', 'darksky.dat'))

Read in the hardware components of the throughput curves. 
The hardware-only is used to calculate sky brightness values, as atmospheric extinction should not be applied to sky brightness. 
It's also useful to combine with other atmospheres, to estimate the effect of atmospheric extinction at different airmass. 

In [ ]:
lsst_hardware = {}
for f in filterlist:
    lsst_hardware[f] = Bandpass()
    lsst_hardware[f].read_throughput_list(['detector.dat', 'lens1.dat', 'lens2.dat', 'lens3.dat', 
                                        'm1.dat', 'm2.dat', 'm3.dat', f'filter_{f}.dat'], 
                                        root_dir=os.path.join(throughputsDir, 'baseline'))

Read in a series of atmospheres with different airmasses. This doesn't include varying atmospheric extinction as e.g. water vapor percentage changes, but it does include a range of atmospheric extinction with the same atmospheric composition. 

In [ ]:
airmass_range = np.arange(1, 2.55, 0.1)

atmosphere = {}
for X in airmass_range:
    atmosphere[X] = Bandpass()
    atmosphere[X].read_throughput(os.path.join(throughputsDir, 'atmos', f'atmos_{int(X*10)}_aerosol.dat'))

---

### Define useful functions ###

Define some functions useful for calculating SNR:
* return a spectra scaled to a specified magnitude in a specified filter (in a standard/reference bandpass)
* return a sky spectra scaled to a specified magnitude in a specified filter
* return the approximate throughput curves at a specified airmass
* convert seeing to the expected FWHM_eff at the specified airmass

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed


def scale_sky(sky_mag, sky_filter, skysed=None):
    if skysed is None:
        skysed = copy.deepcopy(darksky)
    fluxNorm = skysed.calc_flux_norm(sky_mag, lsst_hardware[sky_filter])
    skysed.multiply_flux_norm(fluxNorm)
    return skysed


def throughput_at_X(airmass):
    diff = np.abs(airmass - airmass_range)
    idx = np.where(diff == diff.min())[0]
    X = airmass_range[idx][0]
    lsst_x = {}
    for f in filterlist:
        wavelen, sb = lsst_hardware[f].multiply_throughputs(atmosphere[X].wavelen, atmosphere[X].sb)
        lsst_x[f] = Bandpass(wavelen=wavelen, sb=sb)
    return lsst_x


seeing_model = SeeingModel()
def seeing_at_X(obs_filter, obs_X, atmospheric_seeing_500nm=None, seeing_zenith=None):
    "Return FWHMeff in `obs_filter` at `obs_X`, providing either the seeing @ zenith (in filter) or the FWHM_500"
    if atmospheric_seeing_500nm is not None:
        idx = filterlist.index(obs_filter)
        fwhm_eff = seeing_model(atmospheric_seeing_500nm, obs_X)['fwhmEff'][idx]
    elif seeing_zenith is not None:
        # assume seeing_zenith is in the filter of choice and just approximate for airmass 
        airmass_correction = np.power(obs_X, 0.6)
        fwhm_eff = 1.16 * seeing_zenith * airmass_correction
    else:
        raise ValueErorr('Must specify atmospheric_seeing_500nm or seeing_zenith')
    return fwhm_eff


def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams


def calc_snr(sed, obs_filter, obs_skymag, obs_fwhm_eff, obs_X, 
             ref_filter, ref_mag, 
             exptime=15, nexp=2, verbose=False):
    obs_sky = scale_sky(obs_skymag, obs_filter)
    lsst_x = throughput_at_X(obs_X)
    scaled_sed = scale_sed(ref_mag, ref_filter, sed)
    photParams = set_photometric_parameters(exptime, nexp)
    snr = signaltonoise.calc_snr_sed(scaled_sed, lsst_x[obs_filter], 
                                    obs_sky, lsst_hardware[obs_filter], 
                                    photParams, fwhm_eff=obs_fwhm_eff, 
                                    verbose=verbose)
    return snr

---

### Calculate SNR ###


The SNR of a source depends on the sky background, seeing, and airmass, as well as the sensitivity of the telescope and camera. Calculation of SNR is described in [SMTN-002](https://smtn-002.lsst.io), and the typical sky, seeing, airmass, and throughput sensitivities are described in more detail in [PSTN-054](https://pstn-054.lsst.io). 

We can put these together with `rubin_sim` to estimate SNR for a given source at a given airmass, seeing, and skybackground. Some helpful functions have been defined above, as well as some helpful or required data. 

For the sky background, we currently do not have a variety of lunar phase sky background SEDs to use; so we'll approximate it by using our "zenith dark sky" SED and just scaling it to have the expected magnitude in the given bandpass. Because we don't have a bandpass-spanning sky SED, the calculations below should be calculated a single bandpass at a time, rescaling the sky SED for each filter/skybrightness value. 

In [ ]:
# Specify filter for observing, airmass, and expected skybrightness.
# The seeing can be specified as either the atmospheric contribution at zenith at 500nm ("FWHM_500")
#  or it can be specified as the expected seeing fwhm_eff at zenith, in the specified bandpass.

obs_filter = 'i'
obs_X = 1.5
obs_skybrightness = 19.8 # magnitudes per sq arcsecond 
fwhm_500 = 0.72

obs_fwhmeff = seeing_at_X(obs_filter, obs_X, atmospheric_seeing_500nm=fwhm_500)

For this illustration, we'll calculate the SNR for all of our sample SEDs above. You could also just pop in your own SED.

In [ ]:
# Specify the brightness expected for the source, in a reference (standard) filter 
# (this does not have to be the same as obs_filter)
ref_mag = 24.0
ref_filter = 'r'

snr = {}
for s in seds:
    seds[s] = scale_sed(ref_mag, ref_filter, seds[s])
    snr[s] = calc_snr(seds[s], obs_filter, obs_skybrightness, obs_fwhmeff, obs_X, 
                      ref_filter, ref_mag, 
                      exptime=15, nexp=2)

pd.DataFrame(snr, index=[f'SNR {obs_filter} for sed with {ref_mag} mag in {ref_filter}']).T

In [ ]:
obs_sky = scale_sky(obs_skybrightness, obs_filter)
lsst_x = throughput_at_X(obs_X)

# A plot for illustration - normalized SED, reference bandpass range, and observing bandpass
plt.figure(figsize=(8,5))
for s in seds:
    plt.plot(seds[s].wavelen, seds[s].flambda*seds[s].wavelen, label=s)
ymax = plt.ylim()[1]
# show ref filter range with a simple definition of 'in_band'
in_band = np.where(lsst_std[ref_filter].sb > 0.01)
plt.fill_betweenx(np.arange(0, ymax, ymax/10), 
                  x1=lsst_std[ref_filter].wavelen[in_band].min(),
                  x2=lsst_std[ref_filter].wavelen[in_band].max(),
                  alpha=0.1, color='r')
# show obs filter 
plt.plot(lsst_x[obs_filter].wavelen, lsst_x[obs_filter].sb * (ymax),#/lsst_x[obs_filter].sb.max()), 
         color='k', linewidth=1,)
in_band = np.where(lsst_x[obs_filter].sb > 0.01)
plt.fill_betweenx(np.arange(0, ymax, ymax/10), 
                  x1=lsst_x[obs_filter].wavelen[in_band].min(),
                  x2=lsst_x[obs_filter].wavelen[in_band].max(),
                  alpha=0.1, color='k')
# show sky
skyscale = calc_neff(obs_fwhmeff, set_photometric_parameters(15, 2).platescale)
plt.plot(obs_sky.wavelen, 
         (obs_sky.flambda * obs_sky.wavelen)/skyscale, 
         label='Sky', color='gray', linestyle=':')
plt.xlabel('Wavelength (nm)')
plt.ylabel('$\lambda F_\lambda$')
plt.xlim(300, 1100)
plt.ylim(0, ymax)
plt.legend(loc=(1.01, 0.1), fancybox=True)
plt.grid(alpha=0.3)

----

In [ ]:
# We can investigate the SNR calculation a little closer for a single object, let's use one of the stars.

s = 'km10_7250.fits_g45'

ref_mag = 23.0
ref_filter = 'r'
obs_filter = 'i'
obs_X = 1.5
obs_skybrightness = 20.48 # magnitudes per sq arcsecond 
fwhm_500 = 0.5

obs_fwhmeff = seeing_at_X(obs_filter, obs_X, atmospheric_seeing_500nm=fwhm_500)

snr[s] = calc_snr(seds[s], obs_filter, obs_skybrightness, obs_fwhmeff, obs_X, 
                  ref_filter, ref_mag, exptime=15, nexp=2, verbose=True)


----

In [ ]:
# Calculate the standard dark sky zenith magnitudes - for reference
darksky_mags = {}
for f in filterlist:
    darksky_mags[f] = darksky.calc_mag(lsst_hardware[f])
pd.DataFrame(darksky_mags, index=['Dark sky magnitudes']).round(2)